# CandidateGPT 🇺🇸

**CandidateGPT** is a python-based chatbot built around eight U.S. presidential candidates to allow a user to ask questions about a candidate's political positions. Each candidate has a corresponding embeddings vector database based on publicly available data found on the candidate's corresponding wikipedia page. A subset of candidates was selected from the Wikipedia page, ["2024_United_States_presidential_election"](https://en.wikipedia.org/wiki/2024_United_States_presidential_election), upon which to build the various vector databases.

## Disclaimer:
⚠️ IMPORTANT: This project, the code in it, and outputs, are NOT IN ANY WAY an endorsement of any of the following candidates, their policies, or beliefs. This project was solely an experiment into using multiple vector databases with embeddings to practice semantic text search querying against OpenAI's `text-davinci-003` completion model for generative AI text generation.

## Use-Cases:
- Utilizing generative AI to increase civic engagement
- Augment a given voter's understandings about the policy positions of their candidates
- Provide concise, right-to-the-point, explanations about complex issues
- Improve AC (Augmented Communication) between voters and the candidates

## Tools used:
- AI API: `OpenAI`
- Data Manipulation Library: `Pandas`
- Development/Documentation: `Jupyter Notebook`
- Model for Completions: `text-davinci-003`
- Model for Embeddings: `text-embedding-ada-002`
- Programming Language: `Python`
- Tokenizer: `tokenizer`
- Data Sources: `Wikipedia`

## Data sources:
- [Joe Biden](https://en.wikipedia.org/wiki/Joe_Biden)
- [Ron DeSantis](https://en.wikipedia.org/wiki/Ron_DeSantis)
- [Nikki Haley](https://en.wikipedia.org/wiki/Nikki_Haley)
- [Robert F. Kennedy Jr.](https://en.wikipedia.org/wiki/Robert_F._Kennedy_Jr.)
- [Mike Pence](https://en.wikipedia.org/wiki/Mike_Pence)
- [Vivek Ramaswamy](https://en.wikipedia.org/wiki/Vivek_Ramaswamy)
- [Donald Trump](https://en.wikipedia.org/wiki/Donald_Trump)
- [Marianne Williamson](https://en.wikipedia.org/wiki/Marianne_Williamson)

## Code Details
- To gather an initial dataset, each candidate's Wikipedia page is loaded into an `embeddings.csv` file with 3 columns:
    - index
    - text
    - embeddings
- To order the dataset based on those embeddings, each `embeddings.csv` file is transformed into a `distances.csv` file with 4 columns:
    - index
    - text
    - embeddings
    - distances
- The distances are what will be used to allow our model to find data that is most relevant as "context" when submitting the Zero Shot Prompt into the generative text model.
- For data wrangling, each candidate's dataset is loaded into a `pandas` dataframe with 4 columns: 
    - index
    - text
    - embeddings
    - distances
- The user then must pass in 2 values to a custom query along with a for-looped context array to OpenAI for custom query completion:
    1. `SELECTED_PARAMS_CANDIDATE`
        - This is a camelcase variable listed in the `candidate_params` Python dictionary
        - This variable is connected to the `SELECTED_PARAMS_CANDIATE_FULLNAME` variable which looks up its corresonding value in the `candiate_lookup` directory for the "fullName" of that candidate
    2. `USER_QUESTION_INPUT` 
        - This is a camelcase variable listed in the `candidate_questions_testing` Python dictionary
        - I used this dictionary for testing, but users could type in their own question as well.

## Scenario Details + Dataset Considerations
- In the Jupyter Notebook code below, it shows 2 presidential candidates Joe Biden and Nikki Haley, being asked about their stances on `discrimination` and `foreign policy`.
    - ❌ Without vector database training the model:
        - Example #1 Pt.1: Nikki Haley (UNTRAINED) - discrimination
            - Couldn't generate a response (lack of context?)
        - Example #1 Pt.2: Nikki Haley (UNTRAINED) - foreign policy
            - Couldn't generate a response (lack of context?)

        - Example #2 Pt.1: Joe Biden (UNTRAINED) - discrimination
            - Answer is very general/broad
        - Example #2 Pt.2: Joe Biden (UNTRAINED) - foreign policy
            - Answer is very general/broad
    - ✅ With vector database training the model:
        - Example #3: Nikki Haley (TRAINED) - discrimination
            - Answer is crisp, clear, and detail-oriented
        - Example #4: Joe Biden (TRAINED) - discrimination
            - Answer is crisp, clear, and detail-oriented

        - Example #5: Nikki Haley (TRAINED) - foreign policy
            - Answer is crisp, clear, and detail-oriented
        - Example #6: Joe Biden (TRAINED) - foreign policy
            - Answer is crisp, clear, and detail-oriented
    
- At the end of the Jupyter Notebook, I created a 
- For this application, I created separate vector databases for each of the candidates for the following reasons:
    - Since OpenAI's models are only trained up until 2021, there needs to be a way to fill in the gaps of the model should there not be data available to return a response
    - Since there isn't already that additional context (because the candidates are running for president in 2024 and the model wouldn't have that yet), we can easily pull data from Wikipedia's API to generate a .csv file for our vector database of each candidate.
    - I then map over the distance-sorted embeddings to put in the context up until the token limit that the model will allow. This is important so that I can send the max amount of data possible to the model for background context so that it can sort it, add it to its schema, and then return a well-worded and articulated response back to the user who asked the question.
    - An assumption I made is that instead of jamming all of the candidates' data into one database, creating separate databases for each candidate would increase reliability of answers and speed of delivery, while cutting down on hallucinations wherever possible. 
        - I assume that as the data grows, it would be too burdonsome to re-create a mega-embeddings file, rather than creating embeddings and related vector embeddings distance files for each candidate in consideration. 
        - Also, since the prompts are designed to be one candidate + one topic per query, there would not be a need to cross-over and compare different candidates' datasets.

## Cost Considerations
- When working with tokens, especially in a testing setting, its easy to forget how many times a prompt was run or sent to the OpenAI API. So far, to run this lab, including the setup, tests, and shipping of this project, the total OpenAI costs have reached $1.91. To navigate around costs, these are some strategies I implemented while I built this project:
    - Tried to print out values, dictionaries, and outputs without using any openai methods as much as possible
    - Tried to test very small inputs to the model when I was testing
    - Kept the OpenAI usage graph open and checked the billing after almost every call to see how various calls impacted price
    - Included a price limit to my organization account to ensure there couldn't be a very large bill all of a sudden

# Step 1: Add in Your OpenAI API Key

You will need the following software packages to run the Jupyter workspace:

In [ ]:
pip install openai pandas tiktoken

Replace "Your API Key" with a valid OpenAI API key value and put it inside of the quotes

In [33]:
import openai
openai.api_key = "YOUR API KEY"

# Step 2: Review Non-Trained Results with OpenAI
In this step, we will review the zero shot prompts fed into OpenAI directly without any vector embeddings or context sent to the model. We need to establish a baseline of the model not knowing "what the candidate would be thinking" without our dataset upon which the model would be relying on to answer the questions.

The prompt we want to be able to pass into the chatbot include 3 parts:
- A base prompt template
- The user's selected candidate
- The user's selected question

We need a way to allow the user to pass in their candidate of choice to ask about, as well as a question about their policy position into a zero shot prompt that will go up to OpenAI and pass back a response. We will not be providing any context for testing purposes, so let's see what OpenAI responses with.

### Example #1: Nikki Haley (UNTRAINED) - Discrimination

In [10]:
TEST_PROMPT1_CONTEXT = {}
TEST_PROMPT1_CANDIDATE = "Nikki Haley"
TEST_PROMPT1_QUESTION = "discrimination or equal opportunity"

test_prompt1_without_data = """
Answer the question based on the context below about the candidate's political stance, and if the question can't be answered based on the context, say "Hmmm...I haven't heard their stance on this. Sorry!"

Context: 

{}

---

Question: "What is {}'s stance on {}?"
Answer:
"""

initial_test_prompt1_answer = openai.Completion.create(
    model="text-davinci-003",
    prompt=test_prompt1_without_data.format(TEST_PROMPT1_CONTEXT, TEST_PROMPT1_CANDIDATE, TEST_PROMPT1_QUESTION),
    max_tokens=150
)["choices"][0]["text"].strip()
print(initial_test_prompt1_answer)

Hmmm...I haven't heard their stance on this. Sorry!


### Example #1 Pt.2: Nikki Haley (UNTRAINED) - Foreign Policy

In [34]:
TEST_PROMPT11_CONTEXT = {}
TEST_PROMPT11_CANDIDATE = "Nikki Haley"
TEST_PROMPT11_QUESTION = "foreign policy"

test_prompt11_without_data = """
Answer the question based on the context below about the candidate's political stance, and if the question can't be answered based on the context, say "Hmmm...I haven't heard their stance on this. Sorry!"

Context: 

{}

---

Question: "What is {}'s stance on {}?"
Answer:
"""

initial_test_prompt11_answer = openai.Completion.create(
    model="text-davinci-003",
    prompt=test_prompt11_without_data.format(TEST_PROMPT11_CONTEXT, TEST_PROMPT11_CANDIDATE, TEST_PROMPT11_QUESTION),
    max_tokens=150
)["choices"][0]["text"].strip()
print(initial_test_prompt11_answer)

Hmmm...I haven't heard their stance on this. Sorry!


### Example #2: Joe Biden (UNTRAINED) - Discrimination

In [35]:
TEST_PROMPT2_CONTEXT = {}
TEST_PROMPT2_CANDIDATE = "Joe Biden"
TEST_PROMPT2_QUESTION = "discrimination or equal opportunity"

test_prompt2_without_data = """
Answer the question based on the context below about the candidate's political stance, and if the  question can't be answered based on the context, say "Hmmm...I haven't heard their stance on this. Sorry!"

Context: 

{}

---

Question: "What is {}'s stance on {}?"
Answer:
"""

initial_test_prompt2_answer = openai.Completion.create(
    model="text-davinci-003",
    prompt=test_prompt2_without_data.format(TEST_PROMPT2_CONTEXT, TEST_PROMPT2_CANDIDATE, TEST_PROMPT2_QUESTION),
    max_tokens=150
)["choices"][0]["text"].strip()
print(initial_test_prompt2_answer)

Joe Biden has advocated for equal opportunity and has publicly denounced discrimination. He has criticized issues such as voter suppression, calling it an affront to democracy, and has worked to earn the support of both sides of the aisle to promote equality.


### Example #2 Pt.2: Joe Biden (UNTRAINED) - Foreign Policy

In [36]:
TEST_PROMPT22_CONTEXT = {}
TEST_PROMPT22_CANDIDATE = "Joe Biden"
TEST_PROMPT22_QUESTION = "foreign policy"

test_prompt22_without_data = """
Answer the question based on the context below about the candidate's political stance, and if the  question can't be answered based on the context, say "Hmmm...I haven't heard their stance on this. Sorry!"

Context: 

{}

---

Question: "What is {}'s stance on {}?"
Answer:
"""

initial_test_prompt22_answer = openai.Completion.create(
    model="text-davinci-003",
    prompt=test_prompt22_without_data.format(TEST_PROMPT22_CONTEXT, TEST_PROMPT22_CANDIDATE, TEST_PROMPT22_QUESTION),
    max_tokens=150
)["choices"][0]["text"].strip()
print(initial_test_prompt22_answer)

Joe Biden believes in upholding strong alliances and restoring America's credibility and influence on the world stage. He believes in the necessity of investing in diplomacy and development and engaging with the world to increase security against threats like terrorism, nuclear proliferation, climate change and cyber warfare.


# Step 3: Review Trained Results with OpenAI and Cosine Distance-Based Vector Database
In this step, we will map our vector embeddings database against questions about that candidate so the user will receive unsupervised-learning OpenAI generated responses trained on our candidate databases.

The prompt we want to be able to pass into the chatbot include 4 parts:
- A base prompt template
- The user's selected candidate
- The user's selected question
- The `distances.csv` file which uses cosine difference to group similar items together

We need a way to allow the user to pass in their candidate of choice to ask about, as well as a question about their policy position into a zero shot prompt that will go up to OpenAI and pass back a response. We will be providing context in our zero shot prompt based on our cosine difference embeddings dataset.

In [27]:
import pandas as pd
import string
import tiktoken
tokenizer = tiktoken.get_encoding("cl100k_base")
token_limit = 4090

candidate_params = {
    "joeBiden": {
        "action": "query",
        "prop": "extracts",
        "exlimit": 1,
        "titles": "Joe_Biden",
        "explaintext": 1,
        "formatversion": 2,
        "format": "json"
    },
    "robertFKennedyJr": {
        "action": "query",
        "prop": "extracts",
        "exlimit": 1,
        "titles": "Robert_F._Kennedy_Jr.",
        "explaintext": 1,
        "formatversion": 2,
        "format": "json"
    },
    "marianneWilliamson": {
        "action": "query",
        "prop": "extracts",
        "exlimit": 1,
        "titles": "Marianne_Williamson",
        "explaintext": 1,
        "formatversion": 2,
        "format": "json"
    },
    "nikkiHaley": {
        "action": "query",
        "prop": "extracts",
        "exlimit": 1,
        "titles": "Nikki_Haley",
        "explaintext": 1,
        "formatversion": 2,
        "format": "json"
    },
    "ronDeSantis": {
        "action": "query",
        "prop": "extracts",
        "exlimit": 1,
        "titles": "Ron_DeSantis",
        "explaintext": 1,
        "formatversion": 2,
        "format": "json"
    },
    "vivekRamaswamy": {
        "action": "query",
        "prop": "extracts",
        "exlimit": 1,
        "titles": "Vivek_Ramaswamy",
        "explaintext": 1,
        "formatversion": 2,
        "format": "json"
    },
    "donaldTrump": {
        "action": "query",
        "prop": "extracts",
        "exlimit": 1,
        "titles": "Donald_Trump",
        "explaintext": 1,
        "formatversion": 2,
        "format": "json"
    },
    "mikePence": {
        "action": "query",
        "prop": "extracts",
        "exlimit": 1,
        "titles": "Mike_Pence",
        "explaintext": 1,
        "formatversion": 2,
        "format": "json"
    }
}

candiate_lookup = {
    "joeBiden": {
        "fullName": "Joe Biden"
    },
    "robertFKennedyJr": {
        "fullName": "Robert F. Kennedy Jr."
    },
    "marianneWilliamson": {
        "fullName": "Marianne Williamson"
    },
    "nikkiHaley": {
        "fullName": "Nikki Haley"
    },
    "ronDeSantis": {
        "fullName": "Ron DeSantis"
    },
    "vivekRamaswamy": {
        "fullName": "Vivek Ramaswamy"
    },
    "donaldTrump": {
        "fullName": "Donald Trump"
    },
    "mikePence": {
        "fullName": "Mike Pence"
    }
}

candidate_questions_testing = {
    "topPoints": "top campaign agenda points",
    "schoolChoice": "school choice",
    "environment": "the environment or climate change",
    "discrimination": "discrimination or equal opportunity",
}

EMBEDDING_MODEL_NAME = "text-embedding-ada-002"
COMPLETION_MODEL_NAME = "text-davinci-003"

USER_QUESTION = """What is {}'s stance on {}?"""

### Example #3: Nikki Haley (TRAINED) - Discrimination

In [21]:
SELECTED_PARAMS3_CANDIDATE = "nikkiHaley"

SELECTED_PARAMS3 = candidate_params[SELECTED_PARAMS3_CANDIDATE]
SELECTED_PARAMS3_CANDIATE_FULLNAME = candiate_lookup[SELECTED_PARAMS3_CANDIDATE]["fullName"]

USER_QUESTION_INPUT3 = candidate_questions_testing["discrimination"]
USER_QUESTION_FORMATTED3 = USER_QUESTION.format(
    SELECTED_PARAMS3_CANDIATE_FULLNAME, USER_QUESTION_INPUT3)

df3 = pd.read_csv(
    "distances/{}_distances.csv".format(SELECTED_PARAMS3_CANDIDATE), index_col=0)

prompt3_with_data = """
Answer the question based on the context below about the candidate's political stance, and if the question can't be answered based on the context, say "Hmmm...I haven't heard their stance on this. Sorry!"

Context: 

{}

---

Question: {}
Answer:
"""

token_count3 = len(tokenizer.encode(prompt3_with_data)) + len(tokenizer.encode(
    USER_QUESTION_FORMATTED3
))

context_list3 = []

for text in df3["text"].values:
    token_count3 += len(tokenizer.encode(text))
    if token_count3 <= token_limit:
        context_list3.append(text)
    else:
        break

prompt3 = prompt3_with_data.format(
    "\n\n###\n\n".join(context_list3),
    USER_QUESTION_FORMATTED3
)
# print(prompt3)

prompt3_answer = openai.Completion.create(
    model=COMPLETION_MODEL_NAME,
    prompt=prompt3,
    max_tokens=150
)

answer3 = prompt3_answer["choices"][0]["text"].strip()
print(answer3)

Nimarata Nikki Haley is a strong advocate for equal opportunity and is vehemently against all forms of discrimination. She has spoken out in support of LGBTQ rights, including supporting a bill in South Carolina that would open up employment protection for LGBTQ individuals. Additionally, she is an outspoken proponent of gender and racial equality, and has pushed for legislation to protect minority groups in her home state of South Carolina.


### Example #4: Joe Biden (TRAINED) - Discrimination

In [22]:
SELECTED_PARAMS4_CANDIDATE = "joeBiden"

SELECTED_PARAMS4 = candidate_params[SELECTED_PARAMS4_CANDIDATE]
SELECTED_PARAMS4_CANDIATE_FULLNAME = candiate_lookup[SELECTED_PARAMS4_CANDIDATE]["fullName"]

USER_QUESTION_INPUT4 = candidate_questions_testing["discrimination"]
USER_QUESTION_FORMATTED4 = USER_QUESTION.format(
    SELECTED_PARAMS4_CANDIATE_FULLNAME, USER_QUESTION_INPUT4)

df4 = pd.read_csv(
    "distances/{}_distances.csv".format(SELECTED_PARAMS4_CANDIDATE), index_col=0)

prompt4_with_data = """
Answer the question based on the context below about the candidate's political stance, and if the question can't be answered based on the context, say "Hmmm...I haven't heard their stance on this. Sorry!"

Context: 

{}

---

Question: {}
Answer:
"""

token_count4 = len(tokenizer.encode(prompt4_with_data)) + len(tokenizer.encode(
    USER_QUESTION_FORMATTED4
))

context_list4 = []

for text in df4["text"].values:
    token_count4 += len(tokenizer.encode(text))
    if token_count4 <= token_limit:
        context_list4.append(text)
    else:
        break

prompt4 = prompt4_with_data.format(
    "\n\n###\n\n".join(context_list4),
    USER_QUESTION_FORMATTED4
)
# print(prompt3)

prompt4_answer = openai.Completion.create(
    model=COMPLETION_MODEL_NAME,
    prompt=prompt4,
    max_tokens=150
)

answer4 = prompt4_answer["choices"][0]["text"].strip()
print(answer4)

Joe Biden strongly supports equal opportunity and opposes any forms of discrimination. He has proposed long-term reforms to ensure that minority and marginalized groups have the same access to education, healthcare, and the political process. He also strongly supports the protection of civil rights and human rights, as seen in his support for the Freedom to Vote Act, John Lewis Voting Rights Act, the American Rescue Plan Act of 2021, and his pledge to codify the protections of Roe v. Wade into federal law.


# Step 4: Implement Repeatable Entries like a ChatBot for Better User Communications
In this step, we will change our `for` loop to instead use a `while` loop so that the user can type their questions again, and again, without needing to edit the string content in the code cells every time.

This allows for more of the experience we would get with a chatbot, such as the question-answer-question-answer, etc. type interactivity.

Note: the candidate name you pass in at the first question MUST match the exact spelling listed in `candidate_params` dictionary, which in this case has 2 valid potential entries:
    - `joeBiden`
    - `nikkiHaley`

### Example #5: While Loop
Code output is candidate Nikki Haley asked about Foreign Policy trained on the vector database in /distances

In [31]:
prompt_WHILE_with_data = """
Answer the question based on the context below about the candidate's political stance, and if the question can't be answered based on the context, say "Hmmm...I haven't heard their stance on this. Sorry!"

Context: 

{}

---

Question: {}
Answer:
"""

while True:
    SELECTED_CANDIDATE_WHILE = input("Enter the name of the candidate (or 'quit' to stop): ")
    if SELECTED_CANDIDATE_WHILE.lower() == 'quit':
        break
    if SELECTED_CANDIDATE_WHILE not in candidate_params:
        print("Invalid candidate name. Please try again.")
        continue

    user_question_WHILE = input("Enter the policy or stance you are interested in learning more about (or 'quit' to stop): ")
    if user_question_WHILE.lower() == 'quit':
        break
    selected_params = candidate_params[SELECTED_CANDIDATE_WHILE]
    selected_candidate_fullname_WHILE = candiate_lookup[SELECTED_CANDIDATE_WHILE]["fullName"]

    user_question_formatted = USER_QUESTION.format(selected_candidate_fullname_WHILE, user_question_WHILE)

    df_WHILE = pd.read_csv("distances/{}_distances.csv".format(SELECTED_CANDIDATE_WHILE), index_col=0)

    token_count_WHILE = len(tokenizer.encode(prompt_WHILE_with_data)) + len(tokenizer.encode(user_question_formatted))

    context_list_WHILE = []

    for text in df_WHILE["text"].values:
        token_count_WHILE += len(tokenizer.encode(text))
        if token_count_WHILE <= token_limit:
            context_list_WHILE.append(text)
        else:
            break

    prompt_WHILE = prompt_WHILE_with_data.format("\n\n###\n\n".join(context_list_WHILE), user_question_formatted)

    prompt_answer_WHILE = openai.Completion.create(model=COMPLETION_MODEL_NAME, prompt=prompt_WHILE, max_tokens=150)

    answer_WHILE = prompt_answer_WHILE["choices"][0]["text"].strip()
    print(answer_WHILE)

Nikki Haley has expressed her support of US interests, particularly in relation to Israel, and has been critical of the UN for its perceived anti-Israel bias. She has also championed the withdrawal of the US from the United Nations Human Rights Council, and has shown her willingness to use military force to respond to further North Korean missile tests. Haley has also criticized Trump's rhetoric which could lead to violent tragedy.


### Example #6: While Loop
Code output is candidate Joe Biden asked about Foreign Policy trained on the vector database in /distances

In [30]:
prompt_WHILE_with_data = """
Answer the question based on the context below about the candidate's political stance, and if the question can't be answered based on the context, say "Hmmm...I haven't heard their stance on this. Sorry!"

Context: 

{}

---

Question: {}
Answer:
"""

while True:
    SELECTED_CANDIDATE_WHILE = input("Enter the name of the candidate (or 'quit' to stop): ")
    if SELECTED_CANDIDATE_WHILE.lower() == 'quit':
        break
    if SELECTED_CANDIDATE_WHILE not in candidate_params:
        print("Invalid candidate name. Please try again.")
        continue

    user_question_WHILE = input("Enter the policy or stance you are interested in learning more about (or 'quit' to stop): ")
    if user_question_WHILE.lower() == 'quit':
        break
    selected_params = candidate_params[SELECTED_CANDIDATE_WHILE]
    selected_candidate_fullname_WHILE = candiate_lookup[SELECTED_CANDIDATE_WHILE]["fullName"]

    user_question_formatted = USER_QUESTION.format(selected_candidate_fullname_WHILE, user_question_WHILE)

    df_WHILE = pd.read_csv("distances/{}_distances.csv".format(SELECTED_CANDIDATE_WHILE), index_col=0)

    token_count_WHILE = len(tokenizer.encode(prompt_WHILE_with_data)) + len(tokenizer.encode(user_question_formatted))

    context_list_WHILE = []

    for text in df_WHILE["text"].values:
        token_count_WHILE += len(tokenizer.encode(text))
        if token_count_WHILE <= token_limit:
            context_list_WHILE.append(text)
        else:
            break

    prompt_WHILE = prompt_WHILE_with_data.format("\n\n###\n\n".join(context_list_WHILE), user_question_formatted)

    prompt_answer_WHILE = openai.Completion.create(model=COMPLETION_MODEL_NAME, prompt=prompt_WHILE, max_tokens=150)

    answer_WHILE = prompt_answer_WHILE["choices"][0]["text"].strip()
    print(answer_WHILE)

Joe Biden has said he is against regime change, but for providing non-military support to opposition movements. He has pledged to end U.S. support for the Saudi Arabian-led intervention in Yemen and to reevaluate the United States' relationship with Saudi Arabia. Biden supports extending the New START arms control treaty with Russia to limit the number of nuclear weapons deployed by both sides. He has spoken about human rights abuses in the Xinjiang region to the Chinese Communist Party leader Xi Jinping, pledging to sanction and commercially restrict Chinese government officials and entities who carry out repression. Biden has also endorsed a change to the Senate filibuster to allow for the passing of the Freedom to Vote Act and John Lewis Voting Rights Act, and he has led the U.S.


## MIT License

### Copyright (c) 2023 @BrianHHough/Tech Stack Playbook®️

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

NO WARRANTIES FOR OUTPUT: The output, responses, or answers provided by this Software, either through direct interaction or indirect use, are provided without any warranties. The owner of the Software is not responsible for any inaccuracies, misinformation, or errors produced in the output, responses, or answers by the Software.

USER RESPONSIBILITY: The user of the Software acknowledges and agrees that the user is solely responsible for the interpretation and use of any output, responses, or answers provided by the Software.